<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4872d0dc3b36ce67e9eb78e9190d7a4156595a019a7bdc35d326e18b6bd6102d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-03 10:42:32
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.41 C
-------------------
Today PnL: -94.58 K (-0.66%)
Current PnL: -24.35 L (-15.96%)
CY Booked + Current PnL: -10.46 L (-6.85%)
-------------------
Total profit:  1.40 L
Total loss:  -25.75 L
-------------------
Total Booked + Current PnL: 16.52 L (13.15%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.82%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.76 L (63.46%)
Deployed:  1.26 C
Current:  1.41 C
CAGR/XIRR %: 6.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,-0.30,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.92,253.0,-0.99,0.59,10.15,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.85,-9.91,11.02,0.02,17330.0,-17302.0,157257.0,139.96,62.0,M-SC,14.19,234.0,-1.00,1.11,36.79,OX40N,NTT,PAINTS
51,MASFIN,398.61,0.49,-2.08,24.64,22.05,23640.0,-2040.0,95940.0,-15.54,60.0,H-SC,6.74,164.0,-0.09,0.68,39.46,XR,ATH,FINANCE
43,ITC,452.00,-0.82,-2.04,12.73,10.44,24959.0,-4073.0,196065.0,-41.56,41.0,X-LC,1.41,5.0,-0.16,1.39,3.41,X40,NTT,FMCG
35,ICICIGI,2252.93,-1.38,4.52,14.99,20.19,27313.0,7886.0,182206.0,-18.14,43.0,X-MC,6.69,68.0,0.29,1.29,20.32,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,10.22,-44.10,202.40,69.05,150557.0,-58680.0,74386.0,-54.63,62.0,H-SC,23.59,140.0,-0.39,0.53,14.72,SR,ATH,IT
13,BSOFT,831.70,3.26,-24.19,98.90,50.79,105455.0,-34025.0,106628.0,-2.59,70.0,H-SC,5.90,151.0,-0.32,0.75,24.15,XR,ATH,IT
85,WIPRO,420.00,1.93,4.59,64.71,72.27,114682.0,7777.0,177225.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.25,11.05,XR,NTT,IT
74,TCS,4389.97,1.49,-13.53,37.95,19.28,113526.0,-46810.0,299146.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.12,10.59,X40,ATH,IT
16,CERA,9475.00,1.28,-28.37,72.95,23.89,91921.0,-49897.0,126006.0,-30.79,34.0,H-SC,7.27,157.0,-0.54,0.89,8.82,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-3.72,-38.46,83.45,12.90,112217.0,-84024.0,134472.0,98.32,48.0,H-SC,12.11,161.0,-0.75,0.95,11.81,OX40N,NTT,FINANCE
24,EASEMYTRIP,26.40,-2.23,-53.92,216.17,45.70,176133.0,-95336.0,81479.0,-15.39,65.0,M-SC,23.35,205.0,-0.54,0.58,17.44,XY24,NTT,TRAVEL
28,GREENPANEL,537.00,-2.15,-35.45,121.76,43.14,143883.0,-64909.0,118169.0,164.86,29.0,M-SC,6.82,240.0,-0.45,0.84,11.15,XY24,NTT,MISC
84,WHIRLPOOL,2270.00,-2.15,-20.80,129.71,81.92,117925.0,-23884.0,90914.0,-53.62,22.0,M-SC,4.95,236.0,-0.20,0.64,7.63,XR,NTT,DURABLES
22,DIXON,18931.72,-2.06,-8.83,33.36,21.58,42622.0,-12375.0,127764.0,-54.06,30.0,X-MC,5.84,56.0,-0.29,0.90,13.11,X40N,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.03,-0.02,106.76,106.72,131651.0,-21.0,123315.0,-50.44,39.0,H-SC,1.72,139.0,-0.00,0.87,23.36,AR,ATH,MISC
37,IEX,219.00,1.25,0.81,47.44,48.63,95624.0,1615.0,201569.0,-32.92,66.0,H-SC,15.08,136.0,0.02,1.43,12.45,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.28,-5.93,121.01,107.91,168076.0,-8750.0,138894.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.85,-9.91,11.02,0.02,17330.0,-17302.0,157257.0,139.96,62.0,M-SC,14.19,234.0,-1.00,1.11,36.79,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.30,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.92,253.0,-0.99,0.59,10.15,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.01,-23.02,47.40,13.47,98404.0,-62064.0,207603.0,-68.31,32.0,H-SC,2.15,158.0,-0.63,1.47,5.56,XY24,NTT,PAINTS
67,SIS,528.00,1.00,-24.53,60.68,21.26,50846.0,-27239.0,83793.0,1976.17,46.0,H-SC,3.91,166.0,-0.54,0.59,13.21,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.93,4.59,64.71,72.27,114682.0,7777.0,177225.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.25,11.05,XR,NTT,IT
37,IEX,219.0,1.25,0.81,47.44,48.63,95624.0,1615.0,201569.0,-32.92,66.0,H-SC,15.08,136.0,0.02,1.43,12.45,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.93,4.59,64.71,72.27,114682.0,7777.0,177225.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.25,11.05,XR,NTT,IT
37,IEX,219.00,1.25,0.81,47.44,48.63,95624.0,1615.0,201569.0,-32.92,66.0,H-SC,15.08,136.0,0.02,1.43,12.45,XR,NTT,MISC
38,INDIAMART,4810.62,0.03,-0.02,106.76,106.72,131651.0,-21.0,123315.0,-50.44,39.0,H-SC,1.72,139.0,-0.00,0.87,23.36,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-1.06,-1.64,37.89,35.63,78106.0,-3436.0,206138.0,-15.41,41.0,H-MC,3.64,119.0,-0.04,1.46,14.76,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.28,-5.93,121.01,107.91,168076.0,-8750.0,138894.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.40,-37.52,116.41,35.20,93582.0,-48280.0,80390.0,5.42,20.0,X-SC,14.23,92.0,-0.52,0.57,0.40,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,1.26,-19.65,84.75,48.45,136544.0,-39392.0,161114.0,-25.46,21.0,X-MC,8.80,64.0,-0.29,1.14,1.26,X40N,ATH,FINANCE
8,AWL,485.00,-0.58,-21.53,95.60,53.50,226373.0,-64959.0,236792.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.67,5.17,XY24,NTT,FMCG
23,DMART,5391.45,-1.38,-7.16,37.99,28.12,74213.0,-15057.0,195350.0,-19.94,29.0,X-LC,4.35,19.0,-0.20,1.38,14.78,X40N,ATH,FMCG
55,PGHH,17907.65,-0.50,-5.61,41.71,33.76,79063.0,-11265.0,189555.0,-33.45,29.0,X-MC,4.52,57.0,-0.14,1.34,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.82,-2.04,12.73,10.44,24959.0,-4073.0,196065.0,-41.56,41.0,X-LC,1.41,5.0,-0.16,1.39,3.41,X40,NTT,FMCG
3,ACC,3906.0,-0.75,-22.76,112.51,64.15,206793.0,-54151.0,183800.0,-55.00,46.0,X-SC,1.45,82.0,-0.26,1.30,3.10,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.28,0.63,24.01,24.79,60851.0,1584.0,253440.0,-4.14,45.0,X-SC,1.89,86.0,0.03,1.79,23.91,X40N,NTT,MISC
8,AWL,485.0,-0.58,-21.53,95.60,53.50,226373.0,-64959.0,236792.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.67,5.17,XY24,NTT,FMCG
36,ICICIPRULI,790.0,-0.99,1.20,29.41,30.97,52603.0,2124.0,178862.0,-22.64,47.0,X-MC,2.17,75.0,0.04,1.26,13.91,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-1.92,-19.63,76.07,41.51,161017.0,-51695.0,211670.0,-4.32,32.0,X-MC,7.29,61.0,-0.32,1.50,0.00,XY25,ATH,FMCG
55,PGHH,17907.65,-0.50,-5.61,41.71,33.76,79063.0,-11265.0,189555.0,-33.45,29.0,X-MC,4.52,57.0,-0.14,1.34,0.00,X40,ATH,FMCG
56,QUESS,424.00,-0.36,-29.71,103.23,42.84,47166.0,-19316.0,45690.0,-53.72,38.0,X-SC,37.41,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
54,PAGEIND,51605.08,-0.16,-8.28,37.45,26.07,56242.0,-13560.0,150180.0,-32.40,30.0,X-MC,10.46,55.0,-0.24,1.06,0.37,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.40,-37.52,116.41,35.20,93582.0,-48280.0,80390.0,5.42,20.0,X-SC,14.23,92.0,-0.52,0.57,0.40,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.36,-29.71,103.23,42.84,47166.0,-19316.0,45690.0,-53.72,38.0,X-SC,37.41,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.59,-47.70,191.20,52.31,145167.0,-69236.0,75924.0,-44.34,34.0,X-SC,5.03,91.0,-0.48,0.54,1.63,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.40,-37.52,116.41,35.20,93582.0,-48280.0,80390.0,5.42,20.0,X-SC,14.23,92.0,-0.52,0.57,0.40,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.06,-8.83,33.36,21.58,42622.0,-12375.0,127764.0,-54.06,30.0,X-MC,5.84,56.0,-0.29,0.90,13.11,X40N,ATH,IT
52,MEDANTA,1486.00,-1.98,0.22,23.80,24.08,30281.0,282.0,127232.0,-7.89,39.0,X-SC,5.84,89.0,0.01,0.90,19.88,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,1.49,-13.53,37.95,19.28,113526.0,-46810.0,299146.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.12,10.59,X40,ATH,IT
41,INFY,2275.00,0.40,6.69,45.16,54.87,150750.0,20926.0,333814.0,-17.09,62.0,X-LC,2.69,2.0,0.14,2.36,15.52,X40,BTT,IT
76,TMPV,600.00,-1.20,-15.31,67.88,42.18,158905.0,-42313.0,234097.0,-24.30,32.0,X-LC,3.13,3.0,-0.27,1.66,1.40,XY24,NTT,AUTO
82,VBL,671.64,-1.16,-3.89,41.16,35.67,124945.0,-12282.0,303560.0,-15.30,57.0,X-LC,5.40,4.0,-0.10,2.15,9.43,X40N,ATH,FMCG
43,ITC,452.00,-0.82,-2.04,12.73,10.44,24959.0,-4073.0,196065.0,-41.56,41.0,X-LC,1.41,5.0,-0.16,1.39,3.41,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.53,-16.19,116.77,81.67,91806.0,-15189.0,78621.0,7066.67,41.0,L-SC,18.69,271.0,-0.17,0.56,52.25,XR,NTT,CERAMICS
51,MASFIN,398.61,0.49,-2.08,24.64,22.05,23640.0,-2040.0,95940.0,-15.54,60.0,H-SC,6.74,164.0,-0.09,0.68,39.46,XR,ATH,FINANCE
13,BSOFT,831.70,3.26,-24.19,98.90,50.79,105455.0,-34025.0,106628.0,-2.59,70.0,H-SC,5.90,151.0,-0.32,0.75,24.15,XR,ATH,IT
70,SURYODAY,216.00,-0.37,-19.26,52.38,23.03,75734.0,-34486.0,144585.0,57.89,49.0,H-SC,9.99,167.0,-0.46,1.02,43.46,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.85,-9.91,11.02,0.02,17330.0,-17302.0,157257.0,139.96,62.0,M-SC,14.19,234.0,-1.00,1.11,36.79,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.85,-9.91,11.02,0.02,17330.0,-17302.0,157257.0,139.96,62.0,M-SC,14.19,234.0,-1.00,1.11,36.79,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,-0.38,2.87,17.57,20.95,45507.0,7234.0,259002.0,1.90,77.0,X-LC,17.37,31.0,0.16,1.83,39.53,X40,ATH,PAINTS
13,BSOFT,831.70,3.26,-24.19,98.90,50.79,105455.0,-34025.0,106628.0,-2.59,70.0,H-SC,5.90,151.0,-0.32,0.75,24.15,XR,ATH,IT
50,LTIM,7201.88,-0.01,12.68,16.85,31.66,46735.0,31207.0,277358.0,8.32,69.0,H-LC,14.93,43.0,0.67,1.96,53.89,X200,ATH,IT
66,SIEMENS,4671.50,-0.62,-10.26,39.87,25.51,66583.0,-19095.0,167000.0,2.41,65.0,H-LC,2.72,49.0,-0.29,1.18,23.07,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.04
1,25,44.79
2,50,75.61


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.41
MC    28.90
LC    25.70
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.59
X40      21.32
X40N     11.91
XR        9.73
AR        9.08
XY25      9.06
OX40N     7.58
X200      1.96
SR        1.03
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.97
X-MC    22.44
X-LC    20.33
M-SC    11.89
X-SC     8.16
H-MC     4.75
H-LC     3.14
L-SC     1.39
M-MC     1.37
M-LC     1.25
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.69,-6.68,41.17
IT,14.15,-12.95,71.27
FINANCE,9.35,-17.34,66.92
MISC,7.44,-23.43,74.00
ELECTRICAL,6.01,-10.38,50.82
PAINTS,6.00,-8.88,25.05
INSURANCE,4.37,-2.19,38.20
PHARMA,3.89,-1.43,33.89
AUTO,3.36,-20.95,72.81


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3136322.0,21
AR,1287642.0,10
XR,1287587.0,13
X40,1022636.0,14
X40N,762808.0,9
OX40N,717833.0,10
XY25,361902.0,6
SR,274948.0,2
MH,77731.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3587655.0,25
M-SC,1376354.0,15
X-MC,1306352.0,16
X-LC,891558.0,11
X-SC,764959.0,8
H-MC,407884.0,3
L-SC,265814.0,3
M-LC,114682.0,1
H-LC,113318.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1249403.0      6
           AR         883844.0      5
M-SC       XY24       783904.0      6
H-SC       XR         771274.0      7
X-MC       X40        461074.0      7
           XY24       390925.0      3
X-LC       X40        379729.0      5
H-SC       OX40N      330455.0      4
M-SC       OX40N      300923.0      5
X-SC       X40N       298886.0      3
           XY24       284240.0      3
H-SC       SR         274948.0      2
X-MC       X40N       264764.0      4
X-LC       XY24       233425.0      2
H-MC       AR         213459.0      2
X-LC       X40N       199158.0      2
H-MC       XY24       194425.0      1
X-MC       XY25       189589.0      2
X-SC       X40        181833.0      2
L-SC       XR         179359.0      2
M-SC       XR         167771.0      2
           AR         123756.0      2
M-LC       XR         114682.0      1
L-SC       OX40N       86455.0      1
X-LC       XY25        79246.0      2
H-SC       MH          77731.0      1
H-LC       AR          66583.0      1
L-MC       XR          54501.0      1
M-MC       XY25        51648.0      1
H-LC       X200        46735.0      1
L-LC       XY25        41419.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
